<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-предобработка-данных" data-toc-modified-id="Загрузка-и-предобработка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и предобработка данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Использование модели МО для классификации комментариев на позитивные и негативные (Tfidf)

**Описание проекта:** 
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 


**Цель проекта:** 
Обучить модель классифицировать комментарии на позитивные и негативные. Дан набор данных с разметкой о токсичности правок. Модель должна быть со значением метрики качества *F1* не меньше 0.75. 


**Описание данных:**
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**План работ:**
1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import re
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

## Подготовка

### Загрузка и предобработка данных

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0], parse_dates=[0])
except:
    try:
        data = pd.read_csv('toxic_comments.csv', index_col=[0], parse_dates=[0])
    except:
        print('Something is wrong')

In [3]:
data.info()
data.head(10)

In [4]:
# Проверка на пропуски
data.isna().sum()

In [5]:
#Поиск дубликатов
print('Количество дубликатов до форматирования:', data.duplicated().sum())

### Подготовка данных

In [6]:
def clear_text(text):
    t = re.sub(r'[^a-zA-Z ]+', ' ', text)
    t = re.sub(r'(?:\n|\r)', ' ', text)
    t = t.lower()
    t = t.split()
    return ' '.join(t)

In [7]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

In [8]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)  
    pos_tags = pos_tag(words)
    
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) 
        for word, pos_tag in pos_tags
    ]
    return ' '.join(lemmatized_words)

In [9]:
data['text'] = data['text'].apply(clear_text)
data['text'].head()

In [10]:
data['text'] = data['text'].apply(lemmatize)
data['text'].head()

In [11]:
stopwords = list(nltk_stopwords.words('english'))

In [12]:
X = data['text']
y = data['toxic'].values
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size = 0.25,
    random_state=RANDOM_STATE
)

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tfidf_train = count_tf_idf.fit_transform(X_train)
tfidf_test = count_tf_idf.transform(X_test)

**Вывод:** Данные соответствуют описанию задачи. Пропуски и дубликаты отсутсвуют. Текст твитов был очищен, лемматизирован, разделен на тренировочную и тестовую выборки, а также векторизирован с помощью Tfidf.

## Обучение

In [14]:
# Игнорирование предупреждений
# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.filterwarnings("ignore", category=UserWarning)


# Создаем пайплайн с моделями
pipe_final = Pipeline([
    ('models', LogisticRegression())  # Модель по умолчанию
])

# Настройка параметров для RandomizedSearchCV
param_grid = [
    # Логистическая регрессия
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__C': [0.1, 1, 10, 100],
        'models__solver': ['lbfgs', 'liblinear']
    },

    # Дерево решений
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(5, 50, 5),
        'models__min_samples_split': [2, 5, 10]
    }
]

# RandomizedSearchCV для поиска лучших параметров
randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='f1',  
    random_state=RANDOM_STATE,
    verbose=3,
    n_iter=50,
    n_jobs=-1
)

# Обучаем модель
randomized_search.fit(tfidf_train, y_train)

In [15]:
f1_scorer = make_scorer(f1_score, average='binary')
y_pred = randomized_search.predict(tfidf_test)
print(f"F1 score on test set: {f1_score(y_test, y_pred)}")

**Вывод:** С помощью кросс-валидации и рандомизированного поиска был отобрана модель LogisticRegression(C=100, random_state=42). 

F1=0.77.

## Выводы
На этапе подготовки было выявлено, что данные соответствуют описанию задачи. Пропуски и дубликаты отсутсвуют. Текст твитов был очищен, лемматизирован, разделен на тренировочную и тестовую выборки, а также векторизирован с помощью Tfidf.
С помощью кросс-валидации и рандомизированного поиска был отобрана модель LogisticRegression(C=100, random_state=42). F1-метрика на тестовых данных составила 0.77, что соответсувет порогу не меньше 0.75.